In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/Users/bhuyash./Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bhuyash./Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class EnhancedToxicityDetector:
    def __init__(self):
        self.model_name = "unitary/toxic-bert"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
        
        self.categories = [
            'IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene',
            'IsHatespeech', 'IsRacist', 'IsSexist',
            'IsHomophobic', 'IsReligiousHate', 'IsRadicalism'
        ]
    
    def analyze_text(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            predictions = torch.sigmoid(outputs.logits)
        
        base_score = float(predictions[0][0])
        hate_score = float(predictions[0][5])
        threat_score = float(predictions[0][3])
        obscene_score = float(predictions[0][2])
        
        results = {
            'IsToxic': base_score > 0.5,
            'IsAbusive': base_score > 0.6,
            'IsThreat': threat_score > 0.5,
            'IsProvocative': base_score > 0.4,
            'IsObscene': obscene_score > 0.5,
            'IsHatespeech': hate_score > 0.5,
            'IsRacist': hate_score > 0.6,
            'IsSexist': hate_score > 0.5,
            'IsHomophobic': hate_score > 0.6,
            'IsReligiousHate': hate_score > 0.6,
            'IsRadicalism': (hate_score + threat_score) / 2 > 0.6
        }
        
        return results

def main():
    detector = EnhancedToxicityDetector()
    while True:
        comment = input("\nEnter comment to analyze (or 'quit' to exit): ")
        if comment.lower() == 'quit':
            break
            
        results = detector.analyze_text(comment)
        print("\nAnalysis Results:")
        print("-" * 50)
        for category, value in results.items():
            print(f"{category}: {'Yes' if value else 'No'}")

if __name__ == "__main__":
    main()


Analysis Results:
--------------------------------------------------
IsToxic: Yes
IsAbusive: Yes
IsThreat: No
IsProvocative: Yes
IsObscene: No
IsHatespeech: No
IsRacist: No
IsSexist: No
IsHomophobic: No
IsReligiousHate: No
IsRadicalism: No

Analysis Results:
--------------------------------------------------
IsToxic: No
IsAbusive: No
IsThreat: No
IsProvocative: No
IsObscene: No
IsHatespeech: No
IsRacist: No
IsSexist: No
IsHomophobic: No
IsReligiousHate: No
IsRadicalism: No

Analysis Results:
--------------------------------------------------
IsToxic: No
IsAbusive: No
IsThreat: No
IsProvocative: No
IsObscene: No
IsHatespeech: No
IsRacist: No
IsSexist: No
IsHomophobic: No
IsReligiousHate: No
IsRadicalism: No


KeyboardInterrupt: Interrupted by user